# Specify which Machine Learning problem are you solving.

Estamos resolviendo un problema de clasificación de datos con SVM, en este problema nos proporcionan dos Datasets: Banknote Authentication y Occupancy Detection. Debemos separar los conjuntos de datos de ser posible cuyo objetivo es determinar si hay personas en una habitación y en el otro caso (Occupancy Detection) debemos verificar la autenticidad de un billete. 

                                                                                                        

# Provide a short summary of the features and the labels you are working on.

* Banknote Authentication: las características son cuatro: varianza, asimetría, curtosis, y la entropía y la etiqueta es la autenticidad de la nota (1 para notas auténticas y 0 para notas falsas). 

* Occupancy Detection: Este conjunto de datos contiene información sobre una habitación : temperatura, humedad, los niveles de CO2 y la luz. Las etiquetas indican si la habitación está ocupada o no.

# Please answer the following questions: 

* Are these datasets linearly separable? 
* Are these datasets randomly chosen and 
* The sample size is enough to guarantee generalization.

* Primero para probar que son linealmente separables vamos a usar una SVM que importamos a continuación:

In [9]:
import numpy as np

class SVM:

    def __init__(self, C = 1.0):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0

    # Hinge Loss Function / Calculation
    def hingeloss(self, w, b, x, y):
        # Regularizer term
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Optimization term
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            # calculating loss
            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]

    def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
        # The number of features in X
        number_of_features = X.shape[1]

        # The number of Samples in X
        number_of_samples = X.shape[0]

        c = self.C

        # Creating ids from 0 to number_of_samples - 1
        ids = np.arange(number_of_samples)

        # Shuffling the samples randomly
        np.random.shuffle(ids)

        # creating an array of zeros
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        # Gradient Descent logic
        for i in range(epochs):
            # Calculating the Hinge Loss
            l = self.hingeloss(w, b, X, Y)

            # Appending all losses 
            losses.append(l)
            
            # Starting from 0 to the number of samples with batch_size as interval
            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                for j in range(batch_initial, batch_initial+ batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = Y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            # Calculating the gradients

                            #w.r.t w 
                            gradw += c * Y[x] * X[x]
                            # w.r.t b
                            gradb += c * Y[x]

                # Updating weights and bias
                w = w - learning_rate * w + learning_rate * gradw
                b = b + learning_rate * gradb
        
        self.w = w
        self.b = b

        return self.w, self.b, losses

    def predict(self, X):
        
        prediction = np.dot(X, self.w[0]) + self.b # w.x + b
        return np.sign(prediction)

Luego importamos el dataset banknote authentication con pandas y dividimos con SKLearn los datos de entrenamiento y de test, en este caso se usa una proporción de 80-20

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt', header=None)


df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'class']

df.iloc[:, -1] = df.iloc[:, -1].replace(0, -1)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Separación de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrena la SVM
svm = SVM()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

#Evalúa el modelo
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9854545454545455


Nos arroja una exactitud del 98% aproximadamente, lo que nos permitiría afirmar que nuestro modelo se comporta muy bien y sugiere que los datos en efecto son separables linealmente.

## Ahora con el dataset Occupancy_data

Para este caso los datasets de test y training ya están separados, por tanto primero importaremos el data set training y entrenaremos la SVM en este paso, se hace necesario eliminar la columna "date" ya que al no ser un entero genera fallos al calcular $$w\cdot x+b$$

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


df = pd.read_csv('datatraining.txt')

df.columns = ['date','Temperature','Humidity','Light','CO2','HumidityRatio','Occupancy']

df = df.drop('date', axis=1)

df.iloc[:, -1] = df.iloc[:, -1].replace(0, -1)

X_train = df.iloc[:, :-1].values
y_train = df.iloc[:, -1].values

# Entrena la SVM
svm = SVM()
svm.fit(X_train, y_train)


(array([[-9.83183029e+01, -8.23239866e+01,  3.06406792e+01,
          1.73685080e+00, -1.19912037e-02]]),
 -395.60299999988507,
 [1.0,
  1464.249911874382,
  235.46067322388228,
  455.0010984457841,
  723.4162462786389,
  1221.1068148237407,
  1308.9205011416352,
  1607.3392143760245,
  1924.5781750629997,
  4024.3543171580654,
  2406.4807467838295,
  2650.876971489093,
  3152.401913567856,
  3102.808056080346,
  3585.5756283836295,
  3492.773958584505,
  3702.009209370621,
  3848.382816148228,
  4009.405098337483,
  4549.497091498484,
  4229.876665946885,
  4319.563793356294,
  4421.495601814251,
  4528.653180394039,
  4575.874806226462,
  4667.757511098342,
  4684.274316944735,
  4938.143136313799,
  4663.70696474785,
  4710.263897460786,
  4738.721170008621,
  4772.619416345627,
  4957.308843122973,
  4949.327689116766,
  4959.493521721722,
  4963.979520694887,
  4887.656137867468,
  4933.308311943864,
  4966.258642575734,
  5026.26594943858,
  5011.532518593254,
  5060.354658308584

Ahora importamos el dataset datatest, aquí dividimos etiquetas y características a mano ya que SKLearn no nos permite coger toda la base de datos para test, siempre pide una proporción en test_size, por ello aquí y también en la celda de código anterior tomamos los valores de la última columna como los y_i y los demás como x_i, quitando la fecha.

In [27]:
df = pd.read_csv('datatest.txt')

df.columns = ['date','Temperature','Humidity','Light','CO2','HumidityRatio','Occupancy']

df = df.drop('date', axis=1)

df.iloc[:, -1] = df.iloc[:, -1].replace(0, -1)

X_test1 = df.iloc[:, :-1].values
y_test1 = df.iloc[:, -1].values

Evaluamos sobre los datos

In [28]:
y_pred = svm.predict(X_test1)

#Evalúa el modelo
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test1, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9711069418386492


Y obtenemos exactitud del 97%, veamos ahora el datatest2:

In [29]:
df = pd.read_csv('datatest2.txt')

df.columns = ['date','Temperature','Humidity','Light','CO2','HumidityRatio','Occupancy']

df = df.drop('date', axis=1)

df.iloc[:, -1] = df.iloc[:, -1].replace(0, -1)

X_test2 = df.iloc[:, :-1].values
y_test2 = df.iloc[:, -1].values

In [30]:
y_pred = svm.predict(X_test2)

#Evalúa el modelo
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test2, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9441140278917145


Para este caso implementamos la misma idea, obteniendo una exactitud del 94%

## Conclusión:

Los datos son linealmente separables, eso nos lo sugiere la exactitud del modelo SVM que implementamos, esto para todas las bases de datos

* Es difícil saber si los dataset fueron escogidos de manera aleatoria puesto que no conocemos el proceso de su construcción, sin embargo para el dataset de banknote authentication usamos SKLearn para tomar datos de training aleatoriamente (el 20%), por tanto en ese caso sí, en el de occupancy_data creo que es imposiblem saberlo al menos con las herramientas que tenemos a la mano porque ya nos dan los datos separados.

* Parece que el tamaño de los dataset es suficiente y la prueba de ellos es que la exactitud con la que está funcionando el modelo sobre los datos no observados el bastante alta.

# Provide an explanation how and why the code is working. You can add comments and/or formal explanations into the notebook.

El código que inicialmente nos dan no es funcional para estos ejemplos porque solo funciona en R^2 y además para datos de R^n es muy lenta la búsqueda de los parámetros (w,b), por lo que aún arreglando ese detalle la ejecución puede ser muy lenta, para arreglarlo prácticamente es necesario hacer la SVM desde cero.

Sobe el código de la SVM implementado funciona por que minimiza la norma de w 

$$\min \frac{1}{2}\|\mathbf{w}\|^2 \text {, such that } y_i\left(\mathbf{x}_i \mathbf{w}-b\right)-1 \geq 0, i=1, \ldots, N \text {. }$$

La loss function que debe minimizar como se describe en The Hundred page Machine Learning es:

$$C\|\mathbf{w}\|^2+\frac{1}{N} \sum_{i=1}^N \max \left(0,1-y_i\left(\mathbf{w} \mathbf{x}_i-b\right)\right)$$

y esta se encuentrea implementada en el código:

```python
def hingeloss(self, w, b, x, y):
        # Regularizer term
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Optimization term
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            # calculating loss
            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]
```

El código usa el método del descenso en gradiente para encontrar ese mínimo y tenemos de esa forma la garantía de que nuestro código funciona.

# Show some examples to illustrate that the method is working properly.

Los ejemplos ya fueron proporcionados cuando se mostró que los datos son linealmente separables, llegamos a tener una exactitud sobre los datos mayor al 90% en cada caso, eso siguiere que el método está funcionando de manera adecuada.


# Provide quantitative evidence for generalization using the provided dataset.

Nuevamente la evidencia cuantitativa la mostramos usando SKLearn, obtuvimos una exactitud de más del 90% en todos los casos: 

```python 
y_pred = svm.predict(X_test1)

#Evalúa el modelo
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test1, y_pred)
print('Accuracy:', accuracy)
```